# Debug

## Bug: 
```
System 2 parameters: {'sigma_u': 9.0, 'sigma_b': 0.060145439998495594, 'rho': 1179.133813806039, 'd': 0.7827636158617441, 'label': 1}
Steady state series saved to data_12_04_2025/mRNA_trajectories_variance_3083_1200/steady_state_trajectories/m_traj_3083.9999999999986_1200.0_9_SS.csv
...
...
Attempt 1/10
Attempt 2/10
Attempt 3/10
Attempt 4/10
Attempt 5/10
Attempt 6/10
Attempt 7/10
Attempt 8/10
Attempt 9/10
Attempt 10/10
No suitable solution found after multiple attempts. Try increasing num_guesses or widening the ranges.
[STRESS] ❌ No suitable solution found.
Attempt 1/10
[NORMAL] ✅ Found: {'rho': 1179.1338138059837, 'sigma_b': 0.060145439998495545, 'd': 0.7827636158617439}
Traceback (most recent call last):
  File "/home/ianyang/stochastic_simulations/experiments/SSA_telegraph_model/var_v_accuracy_plot/var_v_accuracy_12_04_2025.py", line 92, in <module>
    "sigma_b": results["stress"]['sigma_b'], 
               ~~~~~~~^^^^^^^^^^
KeyError: 'stress'
```

## Issue: simulation.mean_var_autocorr.find_parameters

In [1]:
# find the variance ratio that went wrong
import numpy as np
from simulation.mean_var_autocorr import find_parameters
import tqdm 
###############################################################################
# 1) Define target mean, variance and autocorrelations, and some parameters to start with
###############################################################################
variance_target_normal = 1200.0  # Fixed normal variance
mu_target = 10.0                 # Mean (same for both)
variance_ratios = np.arange(0.1, 3.0, 0.01)

# this is the ratio that went wrong
current_ratio = variance_ratios[248]
print(current_ratio)

autocorr_target = 0.5

parameters = {
    "stress": {"sigma_u": 18.0},
    "normal": {"sigma_u": 9.0}
}

###############################################################################
# 2) Loop over different variance ratios
###############################################################################
for ratio in tqdm.tqdm(variance_ratios[248:], desc="Running Variance Ratio Simulations"):
    # For the stress condition, we define variance_target_stress by ratio
    variance_target_stress = ratio * variance_target_normal

    # store results for each condition
    results = {}
    for condition, param in parameters.items():
        # Decide which variance to use for this condition
        if condition == "normal":
            var_for_condition = variance_target_normal
        else:  # condition == "stress"
            var_for_condition = variance_target_stress

        try:
            # Fix all three statistical properties
            rho, sigma_b, d = find_parameters(
                param, mu_target, var_for_condition, autocorr_target, 
                rho_range = (1,1000) , num_guesses=1000 # debug
            )

            results[condition] = {"rho": rho, "sigma_b": sigma_b, "d": d}
            print(f"[{condition.upper()}] ✅ Found: {results[condition]}")
        except ValueError as e:
            print(f'{e}')
            print(f"[{condition.upper()}] ❌ No suitable solution found.")


    # update parameter sets 
    parameter_sets = [
        {"sigma_u": parameters["stress"]["sigma_u"], 
        "sigma_b": results["stress"]['sigma_b'], 
        "rho": results["stress"]['rho'], 
        "d": results["stress"]['d'], 
        "label": 0},
        
        {"sigma_u": parameters["normal"]["sigma_u"], 
        "sigma_b": results["normal"]['sigma_b'], 
        "rho": results["normal"]['rho'], 
        "d": results["normal"]['d'], 
        "label": 1}
    ]

    # Output the results
    print("Updated Parameter Sets:", parameter_sets)

    # Simulation parameters
    min_d = min(pset['d'] for pset in parameter_sets)
    steady_state_time = int(10 / min_d)
    time_points = np.arange(0, 144.0, 1.0)
    extended_time_points = np.arange(
        time_points[0],
        len(time_points) + steady_state_time,
        time_points[1] - time_points[0]
    )
    size = 200
    num_iterations = 10

2.5799999999999987


Running Variance Ratio Simulations:   0%|          | 0/42 [00:00<?, ?it/s]

Attempt 1/10


/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:32: RuntimeWarning: invalid value encountered in multiply
  ACmRNA_eq = sp.exp(-d * t) * (
/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:33: RuntimeWarning: invalid value encountered in multiply
  d * sp.exp((d - sigma_u - sigma_b) * t) * rho * sigma_u
/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:164: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  solution = fsolve(
/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:164: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last five Jacobian evaluations.
  solution = fsolve(
/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:164: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  solution = fsolve(


Attempt 2/10
[STRESS] ✅ Found: {'rho': 5795.902957821557, 'sigma_b': 0.022843952095094676, 'd': 0.7346306158352103}
Attempt 1/10


Running Variance Ratio Simulations:   2%|▏         | 1/42 [04:00<2:44:20, 240.50s/it]

[NORMAL] ✅ Found: {'rho': 1179.1338138131455, 'sigma_b': 0.06014543999856846, 'd': 0.7827636158617561}
Updated Parameter Sets: [{'sigma_u': 18.0, 'sigma_b': 0.022843952095094676, 'rho': 5795.902957821557, 'd': 0.7346306158352103, 'label': 0}, {'sigma_u': 9.0, 'sigma_b': 0.06014543999856846, 'rho': 1179.1338138131455, 'd': 0.7827636158617561, 'label': 1}]
Attempt 1/10


Running Variance Ratio Simulations:   2%|▏         | 1/42 [04:25<3:01:22, 265.44s/it]


KeyboardInterrupt: 

In [ ]:
###### var_v_accuracy_12_04_2025.py ######
#!/usr/bin/env python3
import numpy as np
import pandas as pd
import os
import tqdm
# from sympy import sqrt
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.decomposition import PCA
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# import sympy as sp
# from sympy import init_printing, solve
# from scipy.optimize import fsolve

# Import your own local modules/functions
from simulation.simulate_telegraph_model import simulate_two_telegraph_model_systems
from stats.report import statistical_report

from utils.load_data import load_and_split_data
from utils.steady_state import save_steady_state
from simulation.mean_var_autocorr import find_parameters
from classifiers.lstm_classifier import lstm_classifier
from classifiers.lstm_conv1d_classifier import lstm_conv1d_classifier
from classifiers.svm_classifier import svm_classifier
from classifiers.random_forest_classifier import random_forest_classifier
from classifiers.logistic_regression_classifier import logistic_regression_classifier
from classifiers.mlp_classifier import mlp_classifier
from classifiers.random_classifier import random_classifier

############# READ THIS ################
# 1. After finetuning the LSTM architercture and hyperparameters, I am now running some classification benchmarks using the new model (saved in the folder 'src/classifiers/lstm_conv1d_classifier.py').: 
# hidden_size      64.000
# num_layers        2.000
# dropout_rate      0.010
# learning_rate     0.001
# batch_size       32.000
# 2. We want ONLY the steady state part of the data for the classification task, we use save_steady_state() to get the steady state part of the data.
# 3. We want to extend the time points to get enough steady state data.

############# READ THIS ################


###############################################################################
# 1) Define target mean, variance and autocorrelations, and some parameters to start with
###############################################################################
variance_target_normal = 1200.0  # Fixed normal variance
mu_target = 10.0                 # Mean (same for both)
variance_ratios = np.arange(0.1, 3.0, 0.01)
autocorr_target = 0.5

parameters = {
    "stress": {"sigma_u": 18.0},
    "normal": {"sigma_u": 9.0}
}

###############################################################################
# 2) Loop over different variance ratios
###############################################################################
for ratio in tqdm.tqdm(variance_ratios, desc="Running Variance Ratio Simulations"):
    # For the stress condition, we define variance_target_stress by ratio
    variance_target_stress = ratio * variance_target_normal

    # store results for each condition
    results = {}
    for condition, param in parameters.items():
        # Decide which variance to use for this condition
        if condition == "normal":
            var_for_condition = variance_target_normal
        else:  # condition == "stress"
            var_for_condition = variance_target_stress

        try:
            # Fix all three statistical properties
            rho, sigma_b, d = find_parameters(
                param, mu_target, var_for_condition, autocorr_target
            )

            results[condition] = {"rho": rho, "sigma_b": sigma_b, "d": d}
            print(f"[{condition.upper()}] ✅ Found: {results[condition]}")
        except ValueError as e:
            print(f'{e}')
            print(f"[{condition.upper()}] ❌ No suitable solution found.")


    # update parameter sets 
    parameter_sets = [
        {"sigma_u": parameters["stress"]["sigma_u"], 
        "sigma_b": results["stress"]['sigma_b'], 
        "rho": results["stress"]['rho'], 
        "d": results["stress"]['d'], 
        "label": 0},
        
        {"sigma_u": parameters["normal"]["sigma_u"], 
        "sigma_b": results["normal"]['sigma_b'], 
        "rho": results["normal"]['rho'], 
        "d": results["normal"]['d'], 
        "label": 1}
    ]

    # Output the results
    print("Updated Parameter Sets:", parameter_sets)

    # Simulation parameters
    min_d = min(pset['d'] for pset in parameter_sets)
    steady_state_time = int(10 / min_d)
    time_points = np.arange(0, 144.0, 1.0)
    extended_time_points = np.arange(
        time_points[0],
        len(time_points) + steady_state_time,
        time_points[1] - time_points[0]
    )
    size = 200
    num_iterations = 10

    ###########################################################################
    # 3) Simulate & Save data
    ###########################################################################
    for i in range(num_iterations):
        df_results = simulate_two_telegraph_model_systems(parameter_sets, extended_time_points, size)

        output_dir = f"data_12_04_2025/mRNA_trajectories_variance_{int(variance_target_stress)}_{int(variance_target_normal)}"
        os.makedirs(output_dir, exist_ok=True)
        # save full time series
        output_file = f"{output_dir}/m_traj_{variance_target_stress}_{variance_target_normal}_{i}.csv"
        df_results.to_csv(output_file, index=False)

        # get only the steady state part of the data
        save_path = f'{output_dir}/steady_state_trajectories/'
        remaining_time_points, steady_state_series = save_steady_state(output_file, parameter_sets, time_points,
                                                                       save_path=save_path,)

        #######################################################################
        # 4) Analysis & classification (optional, can be done post-hoc)
        #######################################################################
        stress_trajectories = steady_state_series[steady_state_series['label'] == 0].iloc[:, 1:].values
        normal_trajectories = steady_state_series[steady_state_series['label'] == 1].iloc[:, 1:].values
        stats = statistical_report(parameter_sets, stress_trajectories, normal_trajectories)

        # read in the steady state data
        steady_state_file = os.path.join(save_path, f"{os.path.splitext(os.path.basename(output_file))[0]}_SS.csv") # this is the file name we saved it as

        # classifiers
        X_train, X_val, X_test, y_train, y_val, y_test = load_and_split_data(steady_state_file, split_val_size=0.2) # we must define split_val_size here to get a validation set
        svm_rbf_accuracy = svm_classifier(X_train, X_test, y_train, y_test)
        svm_linear_accuracy = svm_classifier(X_train, X_test, y_train, y_test, svm_kernel='linear')
        rf_accuracy = random_forest_classifier(X_train, X_test, y_train, y_test)
        log_reg_accuracy = logistic_regression_classifier(X_train, X_test, y_train, y_test)
        mlp_accuracy = mlp_classifier(X_train, X_val, X_test, y_train, y_val, y_test, epochs=100)
        random_accuracy = random_classifier(y_test)
        lstm_accuracy = lstm_classifier(X_train, X_val, X_test, y_train, y_val, y_test, epochs=50)
        lstm_conv1d_accuracy = lstm_conv1d_classifier(X_train, X_val, X_test, y_train, y_val, y_test, epochs=50)

        df_acc_results = pd.DataFrame({
            "Parameter Sets": [parameter_sets],
            "Stats": [stats],
            "Variance Ratio": [ratio],
            "SVM (rbf) Accuracy": [svm_rbf_accuracy],
            "SVM (linear) Accuracy": [svm_linear_accuracy],
            "Random Forest Accuracy": [rf_accuracy],
            "Logistic Regression Accuracy": [log_reg_accuracy],
            "MLP Accuracy": [mlp_accuracy],
            "Random Classifier Accuracy": [random_accuracy],
            "LSTM Accuracy": [lstm_accuracy],
            "LSTM Conv1D Accuracy": [lstm_conv1d_accuracy]
        })

        # Save the accuracy results to a CSV file
        if not os.path.isfile("data_12_04_2025/accuracy_results_12_04_2025.csv"):
            df_acc_results.to_csv("data_12_04_2025/accuracy_results_12_04_2025.csv", index=False)
        else:
            df_acc_results.to_csv("data_12_04_2025/accuracy_results_12_04_2025.csv", mode='a', header=False, index=False)